# Assignment 3: Build a seq2seq model for machine translation.

### Name: Aishwarya Bhethanabotla

### Task: Change LSTM model to Bidirectional LSTM Model and Translate English to Spanish

### Due Date: Monday, April 21th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue.
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint:

To implement ```Bi-LSTM```, you will need the following code to build the encoder **in Section 3**. Do NOT use Bi-LSTM for the decoder. But there are other codes **you need to modify** to make it work.

In [ ]:
# from keras.layers import Bidirectional, Concatenate

# encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True,
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [9]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [10]:
# e.g., filename = 'Data/deu.txt'
filename = '/dev/spa.txt'


# e.g., n_train = 20000
n_train = 20000

In [11]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [12]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[we are here] => [estamos aqui]
[we ate eggs] => [hemos comido huevos]
[we ate eggs] => [comimos huevos]
[we broke up] => [nos separamos]
[we broke up] => [lo dejamos]
[we broke up] => [rompimos]
[we can help] => [podemos ayudar]
[we can help] => [nosotros podemos ayudar]
[we can meet] => [podemos encontrarnos]
[we can meet] => [podemos vernos]


In [13]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [14]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 48


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length,
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length,
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 48)
shape of target_token_index: 29


In [16]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices.

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [17]:
target_texts[100]

'\tentendiste\n'

In [18]:
decoder_input_seq[100, :]

array([ 6,  2,  9,  8,  2,  9, 15, 11,  5,  8,  2,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [19]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq,
                                    max_decoder_seq_length,
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 48, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- **You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.**

- Build encoder, decoder, and connect the two modules to get "model".

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return:

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [20]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Concatenate
from tensorflow.keras.models import Model

latent_dim = 256

# inputs
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')

# wrap a single LSTM in Bidirectional
encoder_bilstm = Bidirectional(
    LSTM(latent_dim, return_state=True, dropout=0.5, name='encoder_lstm'),
    name='bidirectional_encoder'
)
# discard sequence output (_) and  keep forward/backward hidden & cell states
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# concatenate forward + backward states to form the initial states
state_h = Concatenate(name='state_h')([forward_h, backward_h])
state_c = Concatenate(name='state_c')([forward_c, backward_c])


encoder_model = Model(
    inputs=encoder_inputs,
    outputs=[state_h, state_c],
    name='encoder'
)




Print a summary and save the encoder network structure to "./encoder.pdf"

In [21]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_enco… │ [(None, 512),     │    583,680 │ encoder_inputs[0… │
│ (Bidirectional)     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_h             │ (None, 512)       │          0 │ bidirectional_en… │
│ (Concatenate)       │                   │            │ bidirectional_en… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ state_c             │ (None, 512)       │          0 │ bidirectional_en… │
│ (Concatenate)       │                   │            │ bidirectional_en… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 583,680 (2.23 MB)

 Trainable params: 583,680 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$
    
    -- The initial conveyor belt $c_t$

- Return:

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [22]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim*2,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim*2,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer (units = latent_dim*2 to match concatenated encoder states)
decoder_lstm = LSTM(
    latent_dim*2,
    return_sequences=True,
    return_state=True,
    dropout=0.5,
    name='decoder_lstm'
)
decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_input_x,
    initial_state=[decoder_input_h, decoder_input_c]
)

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(
    inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
    outputs=[decoder_outputs, state_h, state_c],
    name='decoder'
)




Print a summary and save the encoder network structure to "./decoder.pdf"

In [23]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_h     │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_c     │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,112,064 │ decoder_input_x[… │
│                     │ 512), (None,      │            │ decoder_input_h[… │
│                     │ 512), (None,      │            │ decoder_input_c[… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │     15,390 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,127,454 (4.30 MB)

 Trainable params: 1,127,454 (4.30 MB)

 Non-trainable params: 0 (0.00 B)

### 3.3. Connect the encoder and decoder

In [24]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x],
              outputs=decoder_pred,
              name='model_training')

In [25]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model


SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input_x     │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 512),     │    583,680 │ encoder_input_x[… │
│ (Functional)        │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,112,064 │ decoder_input_x[… │
│                     │ 512), (None,      │            │ encoder[0][0],    │
│                     │ 512), (None,      │            │ encoder[0][1]     │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │     15,390 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,711,134 (6.53 MB)

 Trainable params: 1,711,134 (6.53 MB)

 Non-trainable params: 0 (0.00 B)

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [ ]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 48, 30)
shape of decoder_target_data(20000, 48, 30)


In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(64, 18, 28))']
  warnings.warn(msg)


248/250 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.2972

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(None, 18, 28))']
  warnings.warn(msg)


250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 1.2951 - val_loss: 1.1519
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.9497 - val_loss: 1.1255
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.8722 - val_loss: 1.0222
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.8392 - val_loss: 0.9905
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.8238 - val_loss: 0.9767
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.8069 - val_loss: 0.9611
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.7981 - val_loss: 0.9506
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.7885 - val_loss: 0.9419
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.7784 - val_loss: 0.9296
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.7704 - val_loss: 0.9264
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.7557 - val_loss: 0.9139
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/st

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [33]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [34]:
import numpy as np




def sample_with_temperature(preds, temperature=0.1):

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)




def decode_sequence(input_seq, temperature=0.1):
    # Encode the input sentence to get initial states
    states_value = encoder_model.predict(input_seq)


    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    decoded_sentence = ''
    stop_condition = False

    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


        output_tokens[0, -1, 0] = 0


        sampled_token_index = sample_with_temperature(output_tokens[0, -1, :], temperature)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Check for end of sentence or even  max length
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update input & the states
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence




In [ ]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq, temperature=0.1)

    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
-
English:        come see me
Spanish (true):  venid a verme
Spanish (pred):  ven   a marar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences



input_sentence = 'I love you'

input_sentence_clean = input_sentence.lower()

#  Map chars to their training indices
seq = [input_token_index.get(char, 0) for char in input_sentence_clean]
seq_padded = pad_sequences([seq],
                           maxlen=max_encoder_seq_length,
                           padding='post')

input_x = onehot_encode(seq_padded,
                        max_encoder_seq_length,
                        num_encoder_tokens)

#  Decode
translated = decode_sequence(input_x, temperature=0.1).strip()


print('source sentence is:     ' + input_sentence)
print('translated sentence is: ' + translated)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
source sentence is:     I love you
translated sentence is: me gusta el


# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively?

- In this section, you need to re-train the model we built in secton 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [28]:
import numpy as np

def preprocess_data(pairs, n_samples=50000):
    clean_pairs = clean_data(pairs)[:n_samples]
    input_texts = clean_pairs[:, 0]
    target_texts = ['\t' + t + '\n' for t in clean_pairs[:, 1]]
    return input_texts, target_texts

def get_max_lengths(inputs, targets):
    return max(map(len, inputs)), max(map(len, targets))

def prepare_sequences(input_texts, target_texts, max_enc_len, max_dec_len):
    enc_seqs, in_tok_idx = text2sequences(max_enc_len, input_texts)
    dec_seqs, tgt_tok_idx = text2sequences(max_dec_len, target_texts)
    return enc_seqs, dec_seqs, in_tok_idx, tgt_tok_idx








n_train = 50000
input_texts, target_texts = preprocess_data(pairs, n_train)
max_enc_len, max_dec_len = get_max_lengths(input_texts, target_texts)

encoder_seqs, decoder_seqs, input_token_index, target_token_index = prepare_sequences(input_texts, target_texts, max_enc_len, max_dec_len)

num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

encoder_input_data = onehot_encode(encoder_seqs, max_enc_len, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_seqs, max_dec_len, num_decoder_tokens)

decoder_targets = np.roll(decoder_seqs, -1, axis=1)
decoder_targets[:, -1] = 0
decoder_target_data = onehot_encode(decoder_targets, max_dec_len, num_decoder_tokens)

indices = np.random.permutation(n_train)
train_idx, val_idx, test_idx = np.split(indices, [40000, 45000])

def split_data(data):
    return data[train_idx], data[val_idx], data[test_idx]

encoder_input_data_tr, encoder_input_data_val, encoder_input_data_test = split_data(encoder_input_data)
decoder_input_data_tr, decoder_input_data_val, decoder_input_data_test = split_data(decoder_input_data)
decoder_target_data_tr, decoder_target_data_val, decoder_target_data_test = split_data(decoder_target_data)






for name, data in {
    "Train encoder": encoder_input_data_tr,
    "Train decoder input": decoder_input_data_tr,
    "Train decoder target": decoder_target_data_tr,
    "Val encoder": encoder_input_data_val,
    "Val decoder input": decoder_input_data_val,
    "Val decoder target": decoder_target_data_val,
    "Test encoder": encoder_input_data_test,
    "Test decoder input": decoder_input_data_test,
    "Test decoder target": decoder_target_data_test,
}.items():
    print(f'{name} shape: {data.shape}')

print(f'Encoder vocab size: {num_encoder_tokens}')
print(f'Decoder vocab size: {num_decoder_tokens}')
print(f'Max encoder length: {max_enc_len}')
print(f'Max decoder length: {max_dec_len}')



Train encoder shape: (40000, 24, 28)
Train decoder input shape: (40000, 68, 30)
Train decoder target shape: (40000, 68, 30)
Val encoder shape: (5000, 24, 28)
Val decoder input shape: (5000, 68, 30)
Val decoder target shape: (5000, 68, 30)
Test encoder shape: (5000, 24, 28)
Test decoder input shape: (5000, 68, 30)
Test decoder target shape: (5000, 68, 30)
Encoder vocab size: 28
Decoder vocab size: 30
Max encoder length: 24
Max decoder length: 68


### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [29]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy'
)

model.fit(
    x=[encoder_input_data_tr, decoder_input_data_tr],
    y=decoder_target_data_tr,
    validation_data=([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
    batch_size=500,
    epochs=50
)





Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(500, 24, 28))']
  warnings.warn(msg)


80/80 ━━━━━━━━━━━━━━━━━━━━ 15s 119ms/step - loss: 2.5041 - val_loss: 1.1195
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - loss: 1.1057 - val_loss: 1.0462
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - loss: 1.0142 - val_loss: 0.9494
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - loss: 0.9449 - val_loss: 0.9322
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - loss: 0.9260 - val_loss: 0.9165
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 114ms/step - loss: 0.9133 - val_loss: 0.8989
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - loss: 0.8944 - val_loss: 0.8848
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - loss: 0.8826 - val_loss: 0.8768
Epoch 9/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - loss: 0.8747 - val_loss: 0.8649
Epoch 10/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 123ms/step - loss: 0.8625 - val_loss: 0.8534
Epoch 11/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - loss: 0.8517 - val_loss: 0.8440
Epoch 12/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 118ms/step -

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.
2. A reasonable should be 0.1-0.3. The higher, the better.

In [35]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import random


target_texts = np.array(target_texts)
input_texts = np.array(input_texts)

input_texts_test = input_texts[test_idx]
target_texts_test = target_texts[test_idx]

# Limit BLEU evaluation to a random subset of 100 test samples
random.seed(78)
sample_indices = random.sample(range(len(input_texts_test)), 100)


smoothing_fn_1 = SmoothingFunction().method7
bleuscores = []

for idx in sample_indices:
    input_seq = encoder_input_data_test[idx:idx+1]
    reference_text = target_texts_test[idx]

    # Skip empty or malformed cases
    if not reference_text or len(reference_text.strip()) < 2:
        bleuscores.append(0.0)
        continue

    decoded_output = decode_sequence(input_seq).strip()

    reference = list(reference_text[1:-1])
    candidate = list(decoded_output)

    if len(candidate) == 0:
        bleuscores.append(0.0)
        continue

    score = sentence_bleu([reference], candidate,
                          weights=(0.1, 0.8, 0.05, 0.05),
                          smoothing_function=smoothing_fn_1)
    bleuscores.append(score)








1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━

In [36]:
average_bleu = sum(bleuscores) / len(bleuscores)
print(f"Average BLEU Score: {average_bleu}")

Average BLEU Score: 0.2777925427795531
